# `clean_url()`: Clean and validate urls

## Introduction

The function `clean_url()` cleans a column containing urls, and extracts the important parameters including cleaned path, queries, scheme,etc. The function `validate_url()` validates either a single url or a column of urls, returning True if the value is valid, and False otherwise.

`clean_url()` function extracts all the important features of the url and creates an additional column containing key value pairs of the parameters, following are the features it extracts:

* scheme (string)
* host (string) 
* cleaned path (string)
* queries (key-value pairs)

Remove Auth tokens: Sometimes we would like to remove certain sensitive information which is usually contained in a url for e.g. access_tokens, user information, etc. `clean_url()` provides us with an option to remove this information with `remove_auth` parameter. The usage of all parameters is explained in depth in the sections below.

Invalid parsing is handled with the `errors` parameter:

* "coerce" (default), then invalid parsing will be set as NaN
* "ignore", then invalid parsing will return the input
* "raise", then invalid parsing will raise an exception

After cleaning, a **report** is printed that provides the following information:

* How many values were cleaned (the value must be transformed)
* How many values could not be parsed
* If `remove_auth` is specified, then displays how many queries were removed from how many rows
* And the data summary: how many values are in the correct format, and how many values are null

The following sections demonstrate the functionality of `clean_url()` and `validate_url()`. 

In [1]:
import pandas as pd
import numpy as np
df = pd.DataFrame({"messy_url":
                   ["random text which is not a url",
                    "http://www.facebookee.com/otherpath?auth=facebookeeauth&token=iwusdkc&not_token=hiThere&another_token=12323423",
                    "https://www.sfu.ca/ficticiouspath?auth=sampletoken1&studentid=1234&loc=van", 
                    "notaurl", 
                    np.nan,
                    None, 
                    "https://www.sfu.ca/ficticiouspath?auth=sampletoken2&studentid=1230&loc=bur", 
                    "", 
                    {"not_a_url" : True}, 
                    "2345678", 
                    345345345,
                    "https://www.sfu.ca/ficticiouspath?auth=sampletoken3&studentid=1231&loc=sur",
                    "https://www.sfu.ca/ficticiouspath?auth=sampletoken1&studentid=1232&loc=van",
                    ]
                  })
df

,messy_url
0,random text which is not a url
1,http://www.facebookee.com/otherpath?auth=faceb...
2,https://www.sfu.ca/ficticiouspath?auth=samplet...
3,notaurl
4,NaN
5,None
6,https://www.sfu.ca/ficticiouspath?auth=samplet...
7,
8,{'not_a_url': True}
9,2345678


## 1. default: `clean_url()`
By default, all the parameters are set to default, `inplace = False`, `split = False`, `remove_auth = False`, `report = True`,`errors = coerce`.

In [2]:
from dataprep.clean import clean_url
df_default = clean_url(df, column="messy_url")
df_default


Url Cleaning report:
        5 values parsed (38.46 %)
        8 values unable to be parsed (61.54 %), set to NaN 
Result contains parsed key-value pairs for 5 (38.46 %) rows (stored in column `messy_url_details`) and 8 null values(61.54 %).
        


,messy_url,messy_url_details
0,random text which is not a url,NaN
1,http://www.facebookee.com/otherpath?auth=faceb...,"{'scheme': 'http', 'host': 'www.facebookee.com..."
2,https://www.sfu.ca/ficticiouspath?auth=samplet...,"{'scheme': 'https', 'host': 'www.sfu.ca', 'mes..."
3,notaurl,NaN
4,NaN,NaN
5,None,NaN
6,https://www.sfu.ca/ficticiouspath?auth=samplet...,"{'scheme': 'https', 'host': 'www.sfu.ca', 'mes..."
7,,NaN
8,{'not_a_url': True},NaN
9,2345678,NaN


We can see that in the new dataframe `df_default` a new column is created `messy_url_details`, this follows the naming convention of `orginal_column_name`**_details** (`messy_url_details` in our case).

Now let us see what one of the cells in `messy_url_details` looks like.

In [3]:
df_default["messy_url_details"][1]

{'scheme': 'http',
 'host': 'www.facebookee.com',
 'messy_url_clean': 'http://www.facebookee.com/otherpath',
 'queries': {'auth': 'facebookeeauth',
  'token': 'iwusdkc',
  'not_token': 'hiThere',
  'another_token': '12323423'}}

## 2. `remove_auth` parameter

Sometimes we need to remove sensitive information when parsing a url, we can do this in the clean_url() function by specifying the `remove_auth` parameter to be True or we can can specify a list of parameters to removed. Hence `remove_auth` can be a `boolean` value or list of strings.

When `remove_auth` is set to the boolean value of `True`, clean_url() looks for auth tokens based on the default list of token names (provided below) and removes them. When `remove_auth` is set to list of strings it creates a union of the user provided list and default list to create a new set of token words to be removed.

`default_list = {
    "access_token",
    "auth_key",
    "auth",
    "password",
    "username",
    "login",
    "token",
    "passcode",
    "access-token",
    "auth-key",
    "authentication",
    "authentication-key",
}
`

Lets have a look at the same dataframe and the two scenerios described above (by looking at the second row).

### a. `remove_auth = True` (boolean)

In [4]:
df_remove_auth_boolean = clean_url(df, column="messy_url", remove_auth=True)
df_remove_auth_boolean["messy_url_details"][1] 


Url Cleaning report:
        5 values parsed (38.46 %)
        8 values unable to be parsed (61.54 %), set to NaN 
Removed 6 auth queries from 5 rows
Result contains parsed key-value pairs for 5 (38.46 %) rows (stored in column `messy_url_details`) and 8 null values(61.54 %).
        


{'scheme': 'http',
 'host': 'www.facebookee.com',
 'messy_url_clean': 'http://www.facebookee.com/otherpath',
 'queries': {'not_token': 'hiThere', 'another_token': '12323423'}}

As we can see queries `auth` & `token` were removed from the result but `not_token` and `another_token` were included, this is because `auth` and `token` were specified in `default_list`. Also notice the **additional line** giving the stats on how many queries were removed from how many rows.

### b. remove_auth = list of string

In [5]:
df_remove_auth_list = clean_url(df, column="messy_url", remove_auth=["another_token"])
df_remove_auth_list["messy_url_details"][1] 


Url Cleaning report:
        5 values parsed (38.46 %)
        8 values unable to be parsed (61.54 %), set to NaN 
Removed 7 auth queries from 5 rows
Result contains parsed key-value pairs for 5 (38.46 %) rows (stored in column `messy_url_details`) and 8 null values(61.54 %).
        


{'scheme': 'http',
 'host': 'www.facebookee.com',
 'messy_url_clean': 'http://www.facebookee.com/otherpath',
 'queries': {'not_token': 'hiThere'}}

As we can see queries `auth`, `token` and `another_token` were removed but `not_token` was included in the result, this is because a new list was created by creating a union of `default_list` and user defined list and queries were removed based on the new combined list

## 3. `split` Parameter
The `split` parameter adds individual columns containing the containing all the extracted features to the given DataFrame.

In [6]:
df_remove_split = clean_url(df, column="messy_url", split=True)
df_remove_split


Url Cleaning report:
        5 values parsed (38.46 %)
        8 values unable to be parsed (61.54 %), set to NaN 
Result contains parsed values for 5(38.46 %) rows and 8 null values(61.54 %).
        


,messy_url,scheme,host,messy_url_clean,queries
0,random text which is not a url,NaN,NaN,NaN,NaN
1,http://www.facebookee.com/otherpath?auth=faceb...,http,www.facebookee.com,http://www.facebookee.com/otherpath,"{'auth': 'facebookeeauth', 'token': 'iwusdkc',..."
2,https://www.sfu.ca/ficticiouspath?auth=samplet...,https,www.sfu.ca,https://www.sfu.ca/ficticiouspath,"{'auth': 'sampletoken1', 'studentid': '1234', ..."
3,notaurl,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
5,None,NaN,NaN,NaN,NaN
6,https://www.sfu.ca/ficticiouspath?auth=samplet...,https,www.sfu.ca,https://www.sfu.ca/ficticiouspath,"{'auth': 'sampletoken2', 'studentid': '1230', ..."
7,,NaN,NaN,NaN,NaN
8,{'not_a_url': True},NaN,NaN,NaN,NaN
9,2345678,NaN,NaN,NaN,NaN


## 4. `inplace` parameter
Replaces the original column with `orginal_column_name`_details.

In [7]:
df_remove_inplace = clean_url(df, column="messy_url", inplace=True)
df_remove_inplace


Url Cleaning report:
        5 values parsed (38.46 %)
        8 values unable to be parsed (61.54 %), set to NaN 
Result contains parsed key-value pairs for 5 (38.46 %) rows (stored in column `messy_url_details`) and 8 null values(61.54 %).
        


,messy_url_details
0,NaN
1,"{'scheme': 'http', 'host': 'www.facebookee.com..."
2,"{'scheme': 'https', 'host': 'www.sfu.ca', 'mes..."
3,NaN
4,NaN
5,NaN
6,"{'scheme': 'https', 'host': 'www.sfu.ca', 'mes..."
7,NaN
8,NaN
9,NaN


## 5. `split` and `inplace`
Replaces the original column with other columns based on the split parameters.

In [8]:
df_remove_inplace_split = clean_url(df, column="messy_url", inplace=True, split=True)
df_remove_inplace_split


Url Cleaning report:
        5 values parsed (38.46 %)
        8 values unable to be parsed (61.54 %), set to NaN 
Result contains parsed values for 5(38.46 %) rows and 8 null values(61.54 %).
        


,scheme,host,messy_url_clean,queries
0,NaN,NaN,NaN,NaN
1,http,www.facebookee.com,http://www.facebookee.com/otherpath,"{'auth': 'facebookeeauth', 'token': 'iwusdkc',..."
2,https,www.sfu.ca,https://www.sfu.ca/ficticiouspath,"{'auth': 'sampletoken1', 'studentid': '1234', ..."
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN
6,https,www.sfu.ca,https://www.sfu.ca/ficticiouspath,"{'auth': 'sampletoken2', 'studentid': '1230', ..."
7,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN


## 6. `errors` parameter
* "coerce" (default), then invalid parsing will be set as NaN
* "ignore", then invalid parsing will return the input
* "raise", then invalid parsing will raise an exception

### a. "coerce" (default)
This is the default value of the parameters, this sets the invalid parsing to NaN.

In [9]:
df_remove_errors_default = clean_url(df, column="messy_url")
df_remove_errors_default


Url Cleaning report:
        5 values parsed (38.46 %)
        8 values unable to be parsed (61.54 %), set to NaN 
Result contains parsed key-value pairs for 5 (38.46 %) rows (stored in column `messy_url_details`) and 8 null values(61.54 %).
        


,messy_url,messy_url_details
0,random text which is not a url,NaN
1,http://www.facebookee.com/otherpath?auth=faceb...,"{'scheme': 'http', 'host': 'www.facebookee.com..."
2,https://www.sfu.ca/ficticiouspath?auth=samplet...,"{'scheme': 'https', 'host': 'www.sfu.ca', 'mes..."
3,notaurl,NaN
4,NaN,NaN
5,None,NaN
6,https://www.sfu.ca/ficticiouspath?auth=samplet...,"{'scheme': 'https', 'host': 'www.sfu.ca', 'mes..."
7,,NaN
8,{'not_a_url': True},NaN
9,2345678,NaN


### b. "ignore"
This sets the value of invalid parsing as the input.

In [10]:
df_remove_errors_ignore = clean_url(df, column="messy_url", errors="ignore")
df_remove_errors_ignore 


Url Cleaning report:
        5 values parsed (38.46 %)
        8 values unable to be parsed (61.54 %), set to their original values 
Result contains parsed key-value pairs for 5 (38.46 %) rows (stored in column `messy_url_details`) and 8 null / not parsable values(61.54 %).
        


,messy_url,messy_url_details
0,random text which is not a url,random text which is not a url
1,http://www.facebookee.com/otherpath?auth=faceb...,"{'scheme': 'http', 'host': 'www.facebookee.com..."
2,https://www.sfu.ca/ficticiouspath?auth=samplet...,"{'scheme': 'https', 'host': 'www.sfu.ca', 'mes..."
3,notaurl,notaurl
4,NaN,NaN
5,None,None
6,https://www.sfu.ca/ficticiouspath?auth=samplet...,"{'scheme': 'https', 'host': 'www.sfu.ca', 'mes..."
7,,
8,{'not_a_url': True},{'not_a_url': True}
9,2345678,2345678


### c. "raise"
This will raise a value error when it encounters an invalid parsing value.

In [11]:
df_remove_errors_raise = clean_url(df, column="messy_url", errors="raise")
df_remove_errors_raise 

ValueError: Unable to parse value random text which is not a url

## 7. Report Parameter
By default it is set to `True`, when set to `False` - will not display the stats pertaining to the cleaned operations performed.

In [12]:
df_remove_auth_boolean = clean_url(df, column="messy_url", remove_auth=True, report=False)
df_remove_auth_boolean

,messy_url,messy_url_details
0,random text which is not a url,NaN
1,http://www.facebookee.com/otherpath?auth=faceb...,"{'scheme': 'http', 'host': 'www.facebookee.com..."
2,https://www.sfu.ca/ficticiouspath?auth=samplet...,"{'scheme': 'https', 'host': 'www.sfu.ca', 'mes..."
3,notaurl,NaN
4,NaN,NaN
5,None,NaN
6,https://www.sfu.ca/ficticiouspath?auth=samplet...,"{'scheme': 'https', 'host': 'www.sfu.ca', 'mes..."
7,,NaN
8,{'not_a_url': True},NaN
9,2345678,NaN


## 8. Validate_url
validate_url() returns True when the input is a valid url. Otherwise it returns False.

In [13]:
from dataprep.clean import validate_url
print(validate_url({"not_a_url" : True}))
print(validate_url(2346789))
print(validate_url("https://www.sfu.ca/ficticiouspath?auth=sampletoken3&studentid=1231&loc=sur"))
print(validate_url("http://www.facebookee.com/otherpath?auth=facebookeeauth&token=iwusdkc&nottoken=hiThere&another_token=12323423"))


False
False
True
True


In [14]:
df = pd.DataFrame({"messy_url":
                   ["random text which is not a url",
                    "http://www.facebookee.com/otherpath?auth=facebookeeauth&token=iwusdkc&nottoken=hiThere&another_token=12323423",
                    "https://www.sfu.ca/ficticiouspath?auth=sampletoken1&studentid=1234&loc=van", 
                    "notaurl", 
                    np.nan,
                    None, 
                    "https://www.sfu.ca/ficticiouspath?auth=sampletoken2&studentid=1230&loc=bur", 
                    "", 
                    {"not_a_url" : True}, 
                    "2345678", 
                    345345345,
                    "https://www.sfu.ca/ficticiouspath?auth=sampletoken3&studentid=1231&loc=sur",
                    "https://www.sfu.ca/ficticiouspath?auth=sampletoken1&studentid=1232&loc=van",
                    ]
                  })

df["validate_url"] = validate_url(df["messy_url"])
df

,messy_url,validate_url
0,random text which is not a url,False
1,http://www.facebookee.com/otherpath?auth=faceb...,True
2,https://www.sfu.ca/ficticiouspath?auth=samplet...,True
3,notaurl,False
4,NaN,False
5,None,False
6,https://www.sfu.ca/ficticiouspath?auth=samplet...,True
7,,False
8,{'not_a_url': True},False
9,2345678,False
